In [3]:
import tensorflow as tf
import pickle
import numpy as np
import os

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [4]:
def load_data(filename):
    """read data from data file."""
    with open (filename, "rb") as f:
        data = pickle.load(f,encoding='bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1 # 归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape, self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0 ## 遍历起始位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        # [0, 1, 2, 3, 4, 5] -> [5, 3, 2, 4, 0 ,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0 
                end_indicator = batch_size
            else:
                raise Exception("have no more example")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]    
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data, batch_labels)

(50000, 3072) (50000,)
(10000, 3072) (10000,)
[[ 0.96862745  0.98431373  0.99215686 ...  0.55294118  0.56078431
   0.67843137]
 [-0.20784314 -0.21568627 -0.21568627 ... -0.38039216 -0.37254902
  -0.38039216]
 [ 1.          1.          1.         ...  0.34901961  0.31764706
   0.3254902 ]
 ...
 [ 0.09803922  0.08235294  0.09019608 ... -0.23137255 -0.20784314
  -0.22352941]
 [-0.4745098  -0.44313725 -0.56862745 ... -0.25490196 -0.23137255
  -0.33333333]
 [ 0.60784314  0.74117647  0.74901961 ...  0.34901961  0.38039216
   0.41960784]] [3 8 1 2 0 8 0 8 7 1]


In [5]:
x = tf.placeholder(tf.float32, [None, 3072])
# [None], eg: [0,6,5,3]
y = tf.placeholder(tf.int64, [None])

# (3072, 10)
w = tf.get_variable('w', [x.get_shape()[-1], 10],
                   initializer=tf.random_normal_initializer(0, 1))

# (10, )
b = tf.get_variable('b', [10],
                   initializer=tf.constant_initializer(0.0))

# [None, 3072] * [3072, 10] = [None, 10]
y_ = tf.matmul(x, w) + b

# mean square loss
"""
# course: 1 + e^x
# api: e^x / sum(e^x)
# [[0.01, 0.9, ..., 0.03], []]
p_y = tf.nn.softmax(y_)
# 5 -> [0,0,0,0,1,0,0,0,0,0]
y_one_hot = tf.one_hot(y, 10, dtype=tf.float32)
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
"""

loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
# y_ -> softmax
# y -> one_hot
#loss = ylog_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,0]
correct_prediction = tf.equal(predict, y)
accuracy= tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

W0712 15:41:52.066634   252 deprecation.py:323] From D:\anaconda3\envs\deep_learning\lib\site-packages\tensorflow\python\ops\losses\losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps =100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, accu_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if i % 500 == 0:
            print('[Train] step: %d, loss: %4.5f, acc:%4.5f'% (i, loss_val, accu_val)) 
        if i % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            
            all_test_acc_val = []
            
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],
                                       feed_dict={
                                           x: test_batch_data,
                                           y: test_batch_labels
                                       })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f'%(i+1, test_acc)) 

[Train] step: 0, loss: 34.16174, acc:0.10000
(10000, 3072) (10000,)
[Test] Step: 1, acc: 0.09550
[Train] step: 500, loss: 17.68311, acc:0.25000
[Train] step: 1000, loss: 7.59999, acc:0.35000
[Train] step: 1500, loss: 13.93012, acc:0.25000
[Train] step: 2000, loss: 11.68686, acc:0.30000
[Train] step: 2500, loss: 12.85641, acc:0.10000
[Train] step: 3000, loss: 11.22273, acc:0.25000
[Train] step: 3500, loss: 6.63492, acc:0.30000
[Train] step: 4000, loss: 9.45456, acc:0.20000
[Train] step: 4500, loss: 7.88599, acc:0.25000
[Train] step: 5000, loss: 5.69158, acc:0.20000
(10000, 3072) (10000,)
[Test] Step: 5001, acc: 0.25950
[Train] step: 5500, loss: 8.10423, acc:0.20000
[Train] step: 6000, loss: 3.33623, acc:0.45000
[Train] step: 6500, loss: 3.99568, acc:0.40000
[Train] step: 7000, loss: 4.60441, acc:0.50000
[Train] step: 7500, loss: 7.55356, acc:0.15000
[Train] step: 8000, loss: 6.18648, acc:0.20000
[Train] step: 8500, loss: 5.94948, acc:0.25000
[Train] step: 9000, loss: 4.91686, acc:0.3500

[Train] step: 76500, loss: 1.91645, acc:0.40000
[Train] step: 77000, loss: 3.11872, acc:0.15000
[Train] step: 77500, loss: 2.06949, acc:0.40000
[Train] step: 78000, loss: 3.81230, acc:0.20000
[Train] step: 78500, loss: 2.22422, acc:0.45000
[Train] step: 79000, loss: 2.95923, acc:0.25000
[Train] step: 79500, loss: 2.45763, acc:0.30000
[Train] step: 80000, loss: 1.43120, acc:0.50000
(10000, 3072) (10000,)
[Test] Step: 80001, acc: 0.32200
[Train] step: 80500, loss: 2.92671, acc:0.25000
[Train] step: 81000, loss: 2.76020, acc:0.15000
[Train] step: 81500, loss: 2.48692, acc:0.35000
[Train] step: 82000, loss: 2.41864, acc:0.40000
[Train] step: 82500, loss: 2.06482, acc:0.45000
[Train] step: 83000, loss: 3.12434, acc:0.40000
[Train] step: 83500, loss: 3.37385, acc:0.10000
[Train] step: 84000, loss: 3.36245, acc:0.35000
[Train] step: 84500, loss: 2.38397, acc:0.30000
[Train] step: 85000, loss: 3.31113, acc:0.20000
(10000, 3072) (10000,)
[Test] Step: 85001, acc: 0.30600
[Train] step: 85500, los